In [ ]:
!cd /content

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"parbatsingh","key":"f1c70a18f5a515f5e734ab363efee1b2"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv ./kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 90% 73.0M/80.9M [00:00<00:00, 249MB/s]
100% 80.9M/80.9M [00:00<00:00, 194MB/s]


In [ ]:
!unzip /content/sentiment140.zip

Archive:  /content/sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
!pip install contractions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.5 MB/s eta 0:00:00


In [ ]:
import locale
import os
import pandas as pd
import numpy as np
import pickle
from keras.layers import TextVectorization
from keras.layers import Input, Embedding, LSTM, GRU, Dense
from keras.models import Sequential
import tensorflow as tf

In [ ]:
from concurrent.futures.thread import ThreadPoolExecutor
import nltk
from multiprocessing import cpu_count
import contractions
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
cpu_count()

2

In [ ]:
data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv",
                   encoding = "latin-1",header = None)

data = data.sample(n = 1000000)

In [ ]:
data.shape

(1000000, 6)

In [ ]:
data.rename(columns = {0: "target",1: "ids",2: "date",3: "flag",4: "user",5: "text"},
            inplace = True)

In [ ]:
data.head()

,target,ids,date,flag,user,text
956847,4,1825326206,Sun May 17 05:16:40 PDT 2009,NO_QUERY,rociocartes,Decided to watch another horror movie tonight....
664704,0,2244589151,Fri Jun 19 14:56:56 PDT 2009,NO_QUERY,RedheadWench,@l1edmyface0ff Lucy bit our neighbor dog's fac...
921687,4,1754243806,Sun May 10 05:33:35 PDT 2009,NO_QUERY,babigrl21,Happy Mothrs Day to all the amazing women who ...
284703,0,1993351206,Mon Jun 01 10:14:15 PDT 2009,NO_QUERY,LowryGirl,"Another phonecall from tea shop, didn't get jo..."
773390,0,2303175880,Tue Jun 23 17:50:04 PDT 2009,NO_QUERY,mkizzle92,ADD + all day lectures - internet = 2 days in....


In [ ]:
reduced_data = data.drop(labels = data.columns[1:5],axis=1)

In [ ]:
reduced_data.head()

,target,text
956847,4,Decided to watch another horror movie tonight....
664704,0,@l1edmyface0ff Lucy bit our neighbor dog's fac...
921687,4,Happy Mothrs Day to all the amazing women who ...
284703,0,"Another phonecall from tea shop, didn't get jo..."
773390,0,ADD + all day lectures - internet = 2 days in....


In [ ]:
def normalize_tweet(tweet_text):

  return tweet_text.lower()

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(normalize_tweet, list(reduced_data["text"])))

In [ ]:
reduced_data.head()

,target,text
956847,4,decided to watch another horror movie tonight....
664704,0,@l1edmyface0ff lucy bit our neighbor dog's fac...
921687,4,happy mothrs day to all the amazing women who ...
284703,0,"another phonecall from tea shop, didn't get jo..."
773390,0,add + all day lectures - internet = 2 days in....


In [ ]:
def expand_contractions(tweet_text):

  return contractions.fix(tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(expand_contractions,list(reduced_data["text"])))

In [ ]:
reduced_data.head()

,target,text
956847,4,decided to watch another horror movie tonight....
664704,0,@l1edmyface0ff lucy bit our neighbor dog's fac...
921687,4,happy mothrs day to all the amazing women who ...
284703,0,"another phonecall from tea shop, did not get j..."
773390,0,add + all day lectures - internet = 2 days in....


In [ ]:
regex_pattern = r'@[a-zA-z0-9 ]+|#[a-zA-Z0-9 ]+|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [ ]:
def remove_noisy_tokens(tweet_text):

  return re.sub(pattern=regex_pattern,repl=" ",string=tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_noisy_tokens,list(reduced_data["text"])))

In [ ]:
def remove_remaining_noisy_tokens(tweet_text):

  return re.sub(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',repl=" ",string=tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_remaining_noisy_tokens,list(reduced_data["text"])))

In [ ]:
def tokenize_tweet_text(tweet_text):

  return word_tokenize(tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(tokenize_tweet_text,list(reduced_data["text"])))

In [ ]:
len(STOP_WORDS)

326

In [ ]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))
len(en_stop_words)

382

In [ ]:
def is_stopword(token):

  return not(token in en_stop_words)

In [ ]:
def remove_stopwords(tweet_text):

  return list(filter(is_stopword, tweet_text))

Here's an example of the result after removing the stop words from a tweet


In [ ]:
with ThreadPoolExecutor(max_workers = 4) as pool:

  reduced_data["text"] = list(pool.map(remove_stopwords, list(reduced_data["text"])))

In wordnet tags are represented as NN, JJ etc, so to convert them to tags like v for verb, a for adjective etc we convert them using the below function

In [ ]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wn.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wn.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wn.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wn.ADV)
    else:
        return (treebank_tag[0],wn.NOUN)

In [ ]:
def get_pos_tag(tweet_text):

  return list(map(get_wnet_pos_tag,pos_tag(tweet_text)))

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_data["text"] = list(pool.map(get_pos_tag, list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

956847    [(decided, v), (watch, n), (horror, n), (movie...
664704     [(face, n), (bleeding, v), (lazy, a), (text, n)]
921687    [(happy, a), (mothrs, n), (day, n), (amazing, ...
284703    [(phonecall, n), (tea, n), (shop, n), (job, n)...
773390    [(add, n), (day, n), (lectures, v), (internet,...
Name: text, dtype: object

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_token(token_pos_tuple):

  if token_pos_tuple == None:
    return ""
  else:
    return lemmatizer.lemmatize(word = token_pos_tuple[0], pos = token_pos_tuple[1])

In [ ]:
def lemmatize_text(tweet_text):

  if len(tweet_text) > 0:
    return list(map(lemmatize_token, tweet_text))
  else:
    return [""]

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(lemmatize_text, list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

956847    [decide, watch, horror, movie, tonight, hallow...
664704                            [face, bleed, lazy, text]
921687    [happy, mothrs, day, amaze, woman, life, beaut...
284703                 [phonecall, tea, shop, job, unhappy]
773390     [add, day, lecture, internet, day, happy, place]
Name: text, dtype: object

Finding max Sequence length

In [ ]:
max_tokens = 30000
max_sequence_len = max(list(reduced_data["text"].apply(lambda x: len(x))))
print(max_sequence_len)

50


In [ ]:
reduced_rows_idx = np.argwhere(np.array(reduced_data["text"].apply(lambda x: len(x))) >= 10)
print(reduced_rows_idx[1:5])

[[13]
 [18]
 [27]
 [50]]


This basically will keep only those rows whose indexes were present above in our data frame

In [ ]:
reduced_data = reduced_data.iloc[reduced_rows_idx.reshape(reduced_rows_idx.shape[0],)]

Here we are combining the tokens of a tweet

In [ ]:
reduced_data["text"] = list(map(lambda x: " ".join(x), list(reduced_data["text"])))

In [ ]:
reduced_data.head()

,target,text
956847,4,decide watch horror movie tonight halloween ac...
269546,0,oh gosh awake oh right study ddlovato awake oh...
183348,0,free psychic reading coyote ugly tonight merch...
982477,4,idea rid letter bbc bitesize morning haha good...
488177,0,ride get bang motorcyclist sit go send worksho...


In [ ]:
reduced_data.shape


(133954, 2)

Although there was no change in shape after the following line but its a good practice

In [ ]:
filtered_reduced_data = reduced_data[reduced_data["text"] != ""]


**bold text** Now lets create a text vectorization layer

In [ ]:
vectorize_layer = TextVectorization(max_tokens =max_tokens, output_sequence_length = max_sequence_len )

This will vectorize the filtered_reduced_data df and also will build the vocabulary automatically

In [ ]:
vectorize_layer.adapt(data= np.array(filtered_reduced_data["text"]))

Uptil now our data has been adapted to vectorize layer, now we will actually convert the text tweets into integer form to be fed into neural net

In [ ]:
def vectorize_tweet(raw_tweet):

  return vectorize_layer(raw_tweet).numpy()

In [ ]:
vectorized_tweets = list(filtered_reduced_data["text"].apply(vectorize_tweet))

In [ ]:
vectorized_tweets[0]


array([ 407,   17, 2151,   66,   35, 6299,  165,  759, 2648, 1606,    1,
       4690,   18,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

# Now we will divide our dataset into training and testing datasets

In [ ]:
vectorized_train_tweets = vectorized_tweets[0:100000]
vectorized_cv_tweets = vectorized_tweets[100000:133000]

tweet_to_be_predicted = vectorized_tweets[0]
tweet_to_be_predicted = np.expand_dims(np.array(tweet_to_be_predicted), axis=0)

print(tweet_to_be_predicted)



[[ 407   17 2151   66   35 6299  165  759 2648 1606    1 4690   18    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]


now we will change the target values from 0 and 4 to 0 and 1

In [ ]:
filtered_reduced_data["target"] = filtered_reduced_data["target"].apply(lambda x: str(x))

In [ ]:
tweet_labels = list(filtered_reduced_data["target"].replace(to_replace = filtered_reduced_data["target"].unique(),
                                                            value = list(range(len(filtered_reduced_data["target"].unique())))))

now divide labels into trian labels and cv labels

In [ ]:
train_labels = tweet_labels[0:100000]
cv_labels = tweet_labels[100000:133000]



# **Now we will create a data generator for training! its so interesting**

In [ ]:
train_mb_size = 1000       #its 1000 in original code
num_epochs = 10
train_size = 100000         #its 200000 in original code

-Here the outside for loop runs for 10 times
-the inner for loop runs for 100 times
 and at each iteration of the inner loop both the lists are assigned hundred
 tweets and labels respectively

-np.expand will simply make the horizontal array into a vertical one

In [ ]:
def train_datagen():

  for _ in range(num_epochs):

    for i in range(train_size//train_mb_size):

      tweets_mb_list = [vectorized_train_tweets[arr_idx] for arr_idx in range(i*train_mb_size, (i+1)*train_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in train_labels[i*train_mb_size:(i+1)*train_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)


In [ ]:
for tweets_mb, tweets_labels in train_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

(1000, 50)
(1000, 1)


In [ ]:
cv_mb_size = 1000
cv_size = 33000


Same as train_datagen only the fact that this would be used for validation

In [ ]:
def cv_datagen():

  for _ in range(num_epochs):

    for i in range(cv_size//cv_mb_size):

      tweets_mb_list = [vectorized_cv_tweets[arr_idx] for arr_idx in range(i*cv_mb_size,(i+1)*cv_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in cv_labels[i*cv_mb_size:(i+1)*cv_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [ ]:
for tweets_mb,tweets_labels in cv_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

(1000, 50)
(1000, 1)


In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2024-02-12 21:07:24--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-02-12 21:07:24--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2024-02-12 21:10:04 (5.16 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]



In [ ]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
max_vocabulary_size = len(vectorize_layer.get_vocabulary())
embedding_output_dim = 50
print(max_vocabulary_size)

30000


In [ ]:
def create_bin_class_rnn():

  rnn_model = Sequential()

  rnn_model.add(Input(shape=(None,), dtype="int64"))
  rnn_model.add(Embedding(input_dim=max_vocabulary_size, output_dim= embedding_output_dim, input_length= max_sequence_len))
  rnn_model.add(LSTM(units=50))
  rnn_model.add(Dense(units=1,activation="sigmoid"))

  return rnn_model

In [ ]:
rnn_model = create_bin_class_rnn()

In [ ]:
rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 50)          1500000   
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1520251 (5.80 MB)
Trainable params: 1520251 (5.80 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
rnn_model.compile(loss= "binary_crossentropy",metrics= ["accuracy",
                                                        tf.keras.metrics.Precision(),
                                                        tf.keras.metrics.Recall()])

In [ ]:
training_data_gen = train_datagen()
cv_data_gen = cv_datagen()

In [ ]:
rnn_model.fit(training_data_gen,
              epochs= num_epochs,
              validation_data= cv_data_gen,
              steps_per_epoch= 100, validation_steps= 10 )

Epoch 1/10
100/100 [==============================] - 44s 408ms/step - loss: 0.6921 - accuracy: 0.5245 - precision_3: 0.5245 - recall_3: 1.0000 - val_loss: 0.6918 - val_accuracy: 0.5256 - val_precision_3: 0.5256 - val_recall_3: 1.0000
Epoch 2/10
 49/100 [=============>................] - ETA: 19s - loss: 0.6922 - accuracy: 0.5232 - precision_3: 0.5232 - recall_3: 1.0000

KeyboardInterrupt: 

In [ ]:
result = rnn_model.predict(tweet_to_be_predicted)
print(result)


1/1 [==============================] - 0s 34ms/step
[[0.0777093]]
